In [2]:
import pandas as pd 
import numpy as np 

In [3]:
df = pd.read_pickle('./data/training_data')

In [4]:
# reset index 
df.reset_index(drop=True,inplace=True)

In [5]:
# helper functions 
def get_ans_text(answer):
    'gets the text of answer from list of dict'
    if len(answer) != 0: # if it has an answer 
        return answer[0]['text']
    else: return 'answer not in text' # if it doesnt have an answer 
# test function
get_ans_text(df['answers'][4000])

'subway systems'

In [6]:
# update values in answers

df['answers']= df['answers'].apply(get_ans_text)

# Keras Code Example 

https://keras.io/examples/nlp/text_extraction_with_bert/

In [8]:
import os
import re
import json
import string
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer, TFBertModel, BertConfig
from transformers import AutoTokenizer, AutoModelForMaskedLM

max_len = 384
configuration = BertConfig()  # default parameters and configuration for BERT

In [15]:
# Save the slow pretrained tokenizer
slow_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased") # load pretrained tokenizer
save_path = "bert_base_uncased/" # name of path 
if not os.path.exists(save_path):
    os.makedirs(save_path) # if the path doesnt exist make it 
slow_tokenizer.save_pretrained(save_path) # save pretrained model to path in directory (created if it did not exist before)

# Load the fast tokenizer from saved file
# load faster? tokenizer from directory created earlier 
bert_tokenizer = BertWordPieceTokenizer("bert_base_uncased/vocab.txt", lowercase=True)

In [9]:
# ALTERNATIVE TO ABOVE CELL USING ALBERT 
tokenizer = AutoTokenizer.from_pretrained("albert-base-v2")


Downloading:   0%|          | 0.00/684 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/760k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

In [10]:
# get train and eval data 
train_data_url = "https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json"
train_path = keras.utils.get_file("train.json", train_data_url)
eval_data_url = "https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json"
eval_path = keras.utils.get_file("eval.json", eval_data_url)

- Go through the JSON file and store every record as a SquadExample object.
- Go through each SquadExample and create x_train, y_train, x_eval, y_eval.

can change tokenizer in this step to experiment? BERT?WORD2VEC?TFIDI? 

In [31]:
bert_tokenizer.encode('What is this going to do').ids

[101, 2054, 2003, 2023, 2183, 2000, 2079, 102]

In [21]:
bert_tokenizer.encode('What is this going to do').ids

[101, 2054, 2003, 2023, 2183, 2000, 2079, 102]

In [29]:
tokenizer.encode('what is this going to do')

[2, 98, 25, 48, 228, 20, 107, 3]

In [30]:
tokenizer?

Signature:     
tokenizer(
    text: Union[str, List[str], List[List[str]]],
    text_pair: Union[str, List[str], List[List[str]], NoneType] = None,
    add_special_tokens: bool = True,
    padding: Union[bool, str, transformers.file_utils.PaddingStrategy] = False,
    truncation: Union[bool, str, transformers.tokenization_utils_base.TruncationStrategy] = False,
    max_length: Union[int, NoneType] = None,
    stride: int = 0,
    is_split_into_words: bool = False,
    pad_to_multiple_of: Union[int, NoneType] = None,
    return_tensors: Union[str, transformers.file_utils.TensorType, NoneType] = None,
    return_token_type_ids: Union[bool, NoneType] = None,
    return_attention_mask: Union[bool, NoneType] = None,
    return_overflowing_tokens: bool = False,
    return_special_tokens_mask: bool = False,
    return_offsets_mapping: bool = False,
    return_length: bool = False,
    verbose: bool = True,
    **kwargs,
) -> transformers.tokenization_utils_base.BatchEncoding
Type:           

In [28]:
tokenizer('what is this going to do',return_offsets_mapping=True,)

{'input_ids': [2, 98, 25, 48, 228, 20, 107, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1], 'offset_mapping': [(0, 0), (0, 4), (5, 7), (8, 12), (13, 18), (19, 21), (22, 24), (0, 0)]}

In [ ]:
tokenizer.

In [11]:
# define SquadExample class 
class SquadExample:
    
    # each SquadExample instance has following attributes 
    def __init__(self, question, context, start_char_idx, answer_text, all_answers):
        self.question = question # question 
        self.context = context # context 
        self.start_char_idx = start_char_idx # start index of answer 
        self.answer_text = answer_text # text version of answer 
        self.all_answers = all_answers # all the answers 
        self.skip = False
    
    # define preprocess method 
    def preprocess(self):
        # get context,question, answer_text, start_char_idx for the SquadExample instance
        context = self.context 
        question = self.question
        answer_text = self.answer_text
        start_char_idx = self.start_char_idx

        # Clean context, answer and question
        # this doesnt do anything? test it out? 
        # this updates values in context,question,answer by splitting and then joining on space? 
        context = " ".join(str(context).split())
        question = " ".join(str(question).split())
        answer = " ".join(str(answer_text).split())

        # Find end character index of answer in context
        end_char_idx = start_char_idx + len(answer) # beginning + length of entire answer gives index of end 
        if end_char_idx >= len(context): # if the index of end of answer is not in the context then skip? 
            self.skip = True
            return

        # Mark the character indexes in context that are in answer
        is_char_in_ans = [0] * len(context) # array of 0's size of context 
        for idx in range(start_char_idx, end_char_idx):
            is_char_in_ans[idx] = 1 # make 1 for every index of context that is part of answer 

        # Tokenize context
        tokenized_context = tokenizer.encode(context) # uses fast encoder from pretrained tokenizer 

        # Find tokens that were created from answer characters
        ans_token_idx = []
        for idx, (start, end) in enumerate(tokenized_context.offsets):
            if sum(is_char_in_ans[start:end]) > 0:
                ans_token_idx.append(idx)

        if len(ans_token_idx) == 0: # if no answer skip 
            self.skip = True
            return

        # Find start and end token index for tokens from answer
        start_token_idx = ans_token_idx[0]
        end_token_idx = ans_token_idx[-1]

        # Tokenize question
        tokenized_question = tokenizer.encode(question)

        # Create inputs
        input_ids = tokenized_context.ids + tokenized_question.ids[1:]
        token_type_ids = [0] * len(tokenized_context.ids) + [1] * len(
            tokenized_question.ids[1:]
        )
        attention_mask = [1] * len(input_ids)

        # Pad and create attention masks.
        # Skip if truncation is needed
        padding_length = max_len - len(input_ids)
        if padding_length > 0:  # pad
            input_ids = input_ids + ([0] * padding_length)
            attention_mask = attention_mask + ([0] * padding_length)
            token_type_ids = token_type_ids + ([0] * padding_length)
        elif padding_length < 0:  # skip
            self.skip = True
            return
        # add these attributes to SqadExample instance after preprocessing 
        self.input_ids = input_ids
        self.token_type_ids = token_type_ids
        self.attention_mask = attention_mask
        self.start_token_idx = start_token_idx
        self.end_token_idx = end_token_idx
        self.context_token_to_char = tokenized_context.offsets


with open(train_path) as f:
    raw_train_data = json.load(f)

with open(eval_path) as f:
    raw_eval_data = json.load(f)

# makes Squad
def create_squad_examples(raw_data):
    squad_examples = []
    for item in raw_data["data"]:
        for para in item["paragraphs"]:
            context = para["context"]
            for qa in para["qas"]:
                question = qa["question"]
                answer_text = qa["answers"][0]["text"]
                all_answers = [_["text"] for _ in qa["answers"]]
                start_char_idx = qa["answers"][0]["answer_start"]
                squad_eg = SquadExample(
                    question, context, start_char_idx, answer_text, all_answers
                )
                squad_eg.preprocess()
                squad_examples.append(squad_eg)
    return squad_examples


def create_inputs_targets(squad_examples):
    dataset_dict = {
        "input_ids": [],
        "token_type_ids": [],
        "attention_mask": [],
        "start_token_idx": [],
        "end_token_idx": [],
    }
    for item in squad_examples:
        if item.skip == False:
            for key in dataset_dict:
                dataset_dict[key].append(getattr(item, key))
    for key in dataset_dict:
        dataset_dict[key] = np.array(dataset_dict[key])
# DEFINES X AND Y FOR use in models 
    x = [
        dataset_dict["input_ids"],
        dataset_dict["token_type_ids"],
        dataset_dict["attention_mask"],
    ]
    y = [dataset_dict["start_token_idx"], dataset_dict["end_token_idx"]]
    return x, y

# use function to create SQUAD EXAMPLE instances for training data 
train_squad_examples = create_squad_examples(raw_train_data)
# GET X_TRAIN Y_TRAIN from SQUAD EXAMPLE instances 
x_train, y_train = create_inputs_targets(train_squad_examples)
print(f"{len(train_squad_examples)} training points created.")

# use function to create SQUAD EXAMPLE instances for eval data 
eval_squad_examples = create_squad_examples(raw_eval_data)
x_eval, y_eval = create_inputs_targets(eval_squad_examples)
print(f"{len(eval_squad_examples)} evaluation points created.")

AttributeError: 'list' object has no attribute 'offsets'

Create the Question-Answering Model using BERT and Functional API



In [1]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

In [10]:
def create_model():
    ## BERT encoder 
    # TRY DIFFERENT ENCODER MODEL? SMALLER ONE! 
    encoder = TFBertModel.from_pretrained("bert-base-uncased")

    ## QA Model
    input_ids = layers.Input(shape=(max_len,), dtype=tf.int32)
    token_type_ids = layers.Input(shape=(max_len,), dtype=tf.int32)
    attention_mask = layers.Input(shape=(max_len,), dtype=tf.int32)
    embedding = encoder(
        input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask
    )[0]

    start_logits = layers.Dense(1, name="start_logit", use_bias=False)(embedding)
    start_logits = layers.Flatten()(start_logits)

    end_logits = layers.Dense(1, name="end_logit", use_bias=False)(embedding)
    end_logits = layers.Flatten()(end_logits)

    start_probs = layers.Activation(keras.activations.softmax)(start_logits)
    end_probs = layers.Activation(keras.activations.softmax)(end_logits)

    model = keras.Model(
        inputs=[input_ids, token_type_ids, attention_mask],
        outputs=[start_probs, end_probs],
    )
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    optimizer = keras.optimizers.Adam(lr=5e-5)
    model.compile(optimizer=optimizer, loss=[loss, loss])
    return model

In [11]:
use_tpu = True
if use_tpu:
    # Create distribution strategy
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)

    # Create model
    with strategy.scope():
        model = create_model()
else:
    model = create_model()

model.summary()

ValueError: Please provide a TPU Name to connect to.